In [87]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [88]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [89]:
tf.keras.backend.clear_session()

In [90]:
subject = 'Amapá - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 -30

In [91]:
data = pd.read_csv('2003_mo_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - Consumo de Cimento (t)
0,2003-1,0.711421,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.035813e+06,356591.003430,10.883143,7.033593e+06,8.514392,10.392
1,2003-2,0.711553,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.046432e+06,356647.711004,10.885206,7.037356e+06,8.508753,6.857
2,2003-3,0.711685,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.057052e+06,356704.418579,10.887268,7.041120e+06,8.503114,7.011
3,2003-4,0.711817,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.067671e+06,356761.126153,10.889331,7.044883e+06,8.497475,7.122
4,2003-5,0.711949,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.078290e+06,356817.833728,10.891394,7.048646e+06,8.491835,5.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.208
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.476
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.236
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.549


In [92]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [93]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amapá - Consumo de Cimento (t), Length: 240, dtype: float64

In [94]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.388995,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.179199,-1.457028,-1.470096,1.047494,1.300799
158,1.332036,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.184540,-1.486109,-1.464204,1.056372,1.292722
159,1.275077,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.189880,-1.515190,-1.458311,1.065250,1.284645
160,1.218118,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.195221,-1.544271,-1.452419,1.074128,1.276568


In [95]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
157     8.322
158    11.373
159     9.651
160    10.855
161    13.970
Name: Amapá - Consumo de Cimento (t), Length: 162, dtype: float64

In [96]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [97]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 13)

In [98]:
# input para treinamento
test_input = input_data.iloc[train_split-window_size:split_index + 1]
test_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
126,0.639896,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.813802,0.876851,-0.305631,0.923476,0.978734
127,0.580720,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.829759,0.846024,-0.368234,0.925918,0.993895
128,0.521544,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.845716,0.815197,-0.430837,0.928361,1.009056
129,0.462368,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.861673,0.784370,-0.493440,0.930803,1.024217
130,0.403191,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.877629,0.753543,-0.556042,0.933246,1.039378
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [99]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test_target

array([12.399, 11.268,  8.741, 12.358, 11.876, 11.713, 11.449,  7.805,
        7.726,  8.516,  8.503, 10.175, 11.775,  9.724, 10.069, 13.93 ,
       11.918, 11.757,  9.691,  7.378,  6.97 ,  7.612,  8.442,  8.835,
       11.251, 11.342, 13.11 , 13.195, 10.053, 12.274])

In [100]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [101]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [102]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [103]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3828960835, 3554664, 3007430860, 3249044424, 1812759655, 623064449, 2290420119, 3330811976, 2684002854, 3120416615, 4036907344, 656455557, 740593904, 744013833, 302258843, 3519436292, 1598680708, 1180044677, 2869536507, 3117556583]


Step: 0 ___________________________________________
val_loss: 5.486601829528809
winner_seed: 3828960835


Step: 1 ___________________________________________
val_loss: 9.081060409545898


Step: 2 ___________________________________________
val_loss: 9.301746368408203


Step: 3 ___________________________________________
val_loss: 7.055719375610352


Step: 4 ___________________________________________
val_loss: 7.257776260375977


Step: 5 ___________________________________________
val_loss: 6.177021503448486


Step: 6 ___________________________________________
val_loss: 5.691068172454834


Step: 7 ___________________________________________
val_loss: 13.88028621673584


Step: 8 ___________________________________________
val_loss: 2.09226131439209
winner

In [104]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 120ms/step - loss: 237.1177 - val_loss: 103.3136
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 150.8288 - val_loss: 53.5098
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 118.4809 - val_loss: 43.1103
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 105.6167 - val_loss: 37.3093
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 97.5714 - val_loss: 32.6902
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 90.9834 - val_loss: 28.6565
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 84.9777 - val_loss: 25.0540
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 79.6415 - val_loss: 21.8045
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 74.6874 - val_loss: 18.8971
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.3530 - 

4/4 [==============================] - 0s 9ms/step - loss: 31.7860 - val_loss: 11.0040
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7744 - val_loss: 10.8513
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7846 - val_loss: 10.7159
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7854 - val_loss: 10.6238
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7868 - val_loss: 10.5490
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7830 - val_loss: 10.5358
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7853 - val_loss: 10.5236
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7875 - val_loss: 10.5402
Epoch 89/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7888 - val_loss: 10.4641
Epoch 90/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7874 - val_loss: 1

4/4 [==============================] - 0s 9ms/step - loss: 31.7900 - val_loss: 11.0442
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7895 - val_loss: 11.0018
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7920 - val_loss: 10.9682
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7825 - val_loss: 11.0849
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7877 - val_loss: 11.1773
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7845 - val_loss: 11.1873
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7937 - val_loss: 11.1221
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7902 - val_loss: 11.1635
Epoch 168/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7855 - val_loss: 11.1488
Epoch 169/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7908 - val

4/4 [==============================] - 0s 9ms/step - loss: 31.7300 - val_loss: 11.1610
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.6989 - val_loss: 10.9954
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.5198 - val_loss: 10.6975
Epoch 242/10000
4/4 [==============================] - 0s 10ms/step - loss: 30.4468 - val_loss: 9.3300
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.9980 - val_loss: 10.9617
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - loss: 29.5946 - val_loss: 12.4210
Epoch 245/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.5053 - val_loss: 13.1850
Epoch 246/10000
4/4 [==============================] - 0s 11ms/step - loss: 25.5181 - val_loss: 14.5765
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.4866 - val_loss: 16.5720
Epoch 248/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.4253 - v

4/4 [==============================] - 0s 9ms/step - loss: 12.5091 - val_loss: 49.3365
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.3576 - val_loss: 49.2257
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.7696 - val_loss: 49.1166
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.5032 - val_loss: 48.7823
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.6593 - val_loss: 48.9620
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.6465 - val_loss: 49.2732
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.6212 - val_loss: 49.6213
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.3071 - val_loss: 49.8272
Epoch 326/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.2943 - val_loss: 50.1701
Epoch 327/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.5787 - va

...

4/4 [==============================] - 0s 10ms/step - loss: 6.6490 - val_loss: 7.4138
Epoch 1277/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.4466 - val_loss: 8.2987
Epoch 1278/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.6833 - val_loss: 9.6815
Epoch 1279/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.9742 - val_loss: 9.4550
Epoch 1280/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.7137 - val_loss: 8.2528
Epoch 1281/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.6958 - val_loss: 7.6919
Epoch 1282/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.5398 - val_loss: 8.3854
Epoch 1283/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.5505 - val_loss: 8.8616
Epoch 1284/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.5391 - val_loss: 9.2001
Epoch 1285/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.7877 - val_loss: 

4/4 [==============================] - 0s 9ms/step - loss: 8.3914 - val_loss: 23.7691
Epoch 1357/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.0827 - val_loss: 25.8856
Epoch 1358/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.5193 - val_loss: 24.8491
Epoch 1359/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2977 - val_loss: 22.9223
Epoch 1360/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.4391 - val_loss: 18.3276
Epoch 1361/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.1590 - val_loss: 13.3828
Epoch 1362/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.7790 - val_loss: 9.1417
Epoch 1363/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.5914 - val_loss: 8.6480
Epoch 1364/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.7650 - val_loss: 9.3468
Epoch 1365/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.2030 - val_los

4/4 [==============================] - 0s 8ms/step - loss: 5.9384 - val_loss: 8.2321
Epoch 1437/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7351 - val_loss: 8.1065
Epoch 1438/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0231 - val_loss: 8.0380
Epoch 1439/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.5820 - val_loss: 7.9444
Epoch 1440/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7053 - val_loss: 7.9519
Epoch 1441/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8863 - val_loss: 8.2324
Epoch 1442/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8954 - val_loss: 8.4283
Epoch 1443/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9829 - val_loss: 8.1390
Epoch 1444/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9779 - val_loss: 8.0695
Epoch 1445/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6805 - val_loss: 7.94

4/4 [==============================] - 0s 7ms/step - loss: 6.4463 - val_loss: 6.7641
Epoch 1517/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.8617 - val_loss: 6.4140
Epoch 1518/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.6345 - val_loss: 6.6069
Epoch 1519/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.5565 - val_loss: 7.3268
Epoch 1520/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.6030 - val_loss: 8.7141
Epoch 1521/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5104 - val_loss: 9.0579
Epoch 1522/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.3845 - val_loss: 7.4307
Epoch 1523/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.7828 - val_loss: 5.9971
Epoch 1524/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.8505 - val_loss: 6.2000
Epoch 1525/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.8816 - val_loss: 6.49

4/4 [==============================] - 0s 7ms/step - loss: 5.1818 - val_loss: 2.4880
Epoch 1597/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.5482 - val_loss: 2.5173
Epoch 1598/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.1808 - val_loss: 2.4084
Epoch 1599/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.4439 - val_loss: 2.1968
Epoch 1600/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6896 - val_loss: 2.0757
Epoch 1601/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.7659 - val_loss: 2.1822
Epoch 1602/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.7594 - val_loss: 2.2917
Epoch 1603/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6346 - val_loss: 2.4569
Epoch 1604/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.0868 - val_loss: 2.9346
Epoch 1605/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6681 - val_loss: 2.73

In [105]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [106]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,11.24029,11.238006,11.204777,11.191368,11.170576,11.156716,11.158519,11.20237,11.20323,11.192187,11.184949,11.213366,11.169781,11.23704,11.273937,11.249581,11.233826,11.212325,11.277628,11.327103,11.332505,11.35515,11.386843,11.416487,11.447555,11.466869,11.469124,11.459573,11.442713,11.387045
Target,12.399,11.268,8.741,12.358,11.876,11.713,11.449,7.805,7.726,8.516,8.503,10.175,11.775,9.724,10.069,13.93,11.918,11.757,9.691,7.378,6.97,7.612,8.442,8.835,11.251,11.342,13.11,13.195,10.053,12.274
Error,1.15871,0.029994,2.463777,1.166632,0.705424,0.556284,0.290482,3.39737,3.47723,2.676188,2.681949,1.038365,0.605219,1.51304,1.204937,2.680419,0.684175,0.544675,1.586628,3.949103,4.362505,3.74315,2.944842,2.581487,0.196554,0.124869,1.640876,1.735427,1.389712,0.886955


In [107]:
display(mae)
display(mape)

1.7338994

0.19326113

In [108]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [109]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[134.35635]] - Target[122.52900000000001]| =  Error: [[11.827354]]; MAPE:[[0.09652698]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[135.4722]] - Target[118.101]| =  Error: [[17.3712]]; MAPE:[[0.14708766]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[68.67288]] - Target[71.22500000000001]| =  Error: [[2.5521164]]; MAPE:[[0.03583175]]


[array([[11.827354]], dtype=float32),
 array([[17.3712]], dtype=float32),
 array([[2.5521164]], dtype=float32)]

10.583557

0.0931488